In [25]:
from pathlib import Path
from IPython.display import Video
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.animation
plt.rcParams['figure.figsize'] = [10, 4]
plt.rcParams['figure.dpi'] = 500
#%matplotlib inline

import MDAnalysis as mda
from MDAnalysis.analysis import rms, diffusionmap, align

import numpy as np
import pandas as pd
from tqdm import tqdm
from numba import njit, prange, jit
from numba_progress import ProgressBar
import cv2 as cv

In [2]:
run_analysis_verb = False
save_verb = True
plot_verb = False

run = 2

if run == 1:
    data_path = '../../analysis/data/run_1/'
    results_path = '../../analysis/results/teo_results/run_1/'
if run == 2:
    data_path = '../../analysis/data/run_2/'
    results_path = '../../analysis/results/teo_results/run_2/'
if run == 3:
    data_path = '../../analysis/data/run_wt/'
    results_path = '../../analysis/results/teo_results/run_wt/'

In [3]:
if run == 1:
    gro = '../../trajectories/8DD9_run_1.gro' # starting frame of trajectory
    xtc = '../../trajectories/8DD9_run_1.xtc'

if run == 2:
    gro = '../../trajectories/8DD9_run_2.gro' # starting frame of trajectory
    xtc = '../../trajectories/8DD9_run_2.xtc'

if run == 3:
    gro = '../../trajectories/8DD9_run_wt.gro' # starting frame of trajectory
    xtc = '../../trajectories/8DD9_run_wt.xtc'

u = mda.Universe(gro, xtc)
nFrames = len(u.trajectory)

# ticks for imshow plot
ticks_arr = np.linspace(0, nFrames + 1, 5, dtype = int)
ticks_arr_new = np.linspace(0, (nFrames+1)/20, 5, dtype = int)

In [4]:
if run == 1 or run == 2 :
        first_chain = u.select_atoms('name CA and (index 1:4689)') 
        second_chain = u.select_atoms('name CA and (index 4811:9499)') 
        binding_sites = ('resid 41 or resid 49 or resid 135 or resid 142 or resid 144 ' \
                'or resid 163:166 or resid 172 or resid 189 or resid 192')
        binding_site1 = f'({binding_sites}) and (index 1:4689)'
        binding_site2 = f'({binding_sites}) and (index 4811:9499)'
if run == 3:
        first_chain = u.select_atoms('name CA and (index 1:4682)') 
        second_chain = u.select_atoms('name CA and (index 4683:9364)')
        chain_resid_ticks = np.linspace(1, 306, 10, dtype=int)
        binding_sites = ('resid 41 or resid 49 or resid 135 or resid 142 or resid 144 ' \
                'or resid 163:166 or resid 172 or resid 189 or resid 192')
        binding_site1 = f'({binding_sites}) and (index 1:4682)'
        binding_site2 = f'({binding_sites}) and (index 4683:9364)'

binding_site1_CA = u.select_atoms(binding_site1 + ' and (name CA)')
binding_site2_CA = u.select_atoms(binding_site2 + ' and (name CA)')

lig1 = "resid 307"
lig2 = "resid 308"
lig1_selection = u.select_atoms(lig1)
lig2_selection = u.select_atoms(lig2)

# RMSD ANALYSIS

In [5]:
if run_analysis_verb:
    R = rms.RMSD(u, u, select='name CA', groupselections=[lig1, lig2, binding_site1, binding_site2], ref_frame=0).run(verbose = True)
    rmsd_df = pd.DataFrame(R.rmsd, columns=['idx', 'time', 'CA', 'lig1', 'lig2', 'pot1', 'pot2'])
    rmsd_df['time'] = rmsd_df.time / 1000 # to get time in nanoseconds
    rmsd_df.to_csv(data_path + 'rmsd.csv', index=False, float_format='%.3f')
else:
    try: rmsd_df = pd.read_csv(data_path + 'rmsd.csv')
    except: print('Error: You have to run the analysis for the first time')

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 6), sharex=True)
ax1.plot(rmsd_df.time, rmsd_df.CA)
ax1.set(ylabel = 'RMSD ($\AA$)', title = 'CA RMSD')
ax1.grid()

ax2.plot(rmsd_df.time, rmsd_df.lig1, label='lig1', linewidth=0.5)
ax2.plot(rmsd_df.time, rmsd_df.lig2, label='lig2', linewidth=0.5)
ax2.set(ylabel = 'RMSD ($\AA$)', title = 'Ligands RMSD')
ax2.legend()
ax2.grid()

ax3.plot(rmsd_df.time, rmsd_df.pot1, label='pot1', linewidth=0.5)
ax3.plot(rmsd_df.time, rmsd_df.pot2, label='pot2', linewidth=0.5)
ax3.set(ylabel = 'RMSD ($\AA$)', title = 'Binding sites RMSD')
ax3.set_xlabel('Time (ns)')
ax3.legend()
#ax3.set_xlim(10, 500)
ax3.grid()
plt.tight_layout()

if save_verb: plt.savefig(results_path + "rmsd/rmsd.png", dpi = 500, bbox_inches = 'tight')
if plot_verb: 
    plt.show()
else: plt.close()

# RMSD MAP & CLUSTERING 

In [ ]:
@njit(nogil=True)
def compute_rmsd(tr1, tr2):
    # Calculate the RMSD between the two frames
    diff = tr1 - tr2
    rmsd = np.sqrt(np.mean(diff**2))
    return rmsd

@njit(nogil=True, parallel=True)
def compute_pairwise_rmsd(traj1, traj2, rmsd_map, progress_proxy):    
    for i in prange(len(traj1)):
        for j in prange(len(traj2)):
            rmsd_map[i, j] = compute_rmsd(traj1[i], traj2[j])
            progress_proxy.update(1)
    return rmsd_map

## LIGAND

In [6]:
if run_analysis_verb:
    # take positions of only the lig 1 atoms
    selection = lig1
    traj = []
    for ts in u.trajectory:
        traj.append(u.select_atoms(selection).positions)
    traj = np.array(traj)

    print("Now starting lig 1 rmsd map calculation")

    with ProgressBar(total=len(traj)*len(traj)) as progress:
        rmsd_map_lig1 = compute_pairwise_rmsd(traj, traj, np.zeros((len(traj), len(traj))), progress)

    # take positions of only the lig 2 atoms
    selection = lig2
    traj = []
    for ts in u.trajectory:
        traj.append(u.select_atoms(selection).positions)
    traj = np.array(traj)

    print("Now starting lig 2 rmsd map calculation")

    with ProgressBar(total=len(traj)*len(traj)) as progress:
        rmsd_map_lig2 = compute_pairwise_rmsd(traj, traj, np.zeros((len(traj), len(traj))), progress)

    df = pd.DataFrame(rmsd_map_lig1).round(3)
    df.columns = [f'{i}' for i in range(nFrames)]
    df.to_parquet(data_path + 'rmsd_map/rmsd_lig1.parquet', index=False, engine = 'pyarrow')

    df = pd.DataFrame(rmsd_map_lig2).round(3)
    df.columns = [f'{i}' for i in range(nFrames)]
    df.to_parquet(data_path + 'rmsd_map/rmsd_lig2.parquet', index=False, engine = 'pyarrow')
else:
    try:
        rmsd_map_lig1 = pd.read_parquet(data_path + 'rmsd_map/rmsd_lig1.parquet')
        rmsd_map_lig2 = pd.read_parquet(data_path + 'rmsd_map/rmsd_lig2.parquet')
    except: print('Error: You have to run the analysis for the first time')

In [ ]:
fig, (ax, ax1) = plt.subplots(1, 2, figsize=(10, 4))
im = ax.imshow(rmsd_map_lig1, cmap='viridis', vmin=0, vmax=15)
ax.set_xlabel('Time [ns]')
ax.set_ylabel('Time [ns]')
ax.set_xticks(ticks_arr, ticks_arr_new.astype(str))
ax.set_yticks(ticks_arr, ticks_arr_new.astype(str))
ax.set_title("Ligand 1")

im1 = ax1.imshow(rmsd_map_lig2, cmap='viridis', vmin=0, vmax=15)
ax1.set_xlabel('Time [ns]')
ax1.set_xticks(ticks_arr, ticks_arr_new.astype(str))
ax1.set_yticks(ticks_arr, ticks_arr_new.astype(str))
ax1.set_title("Ligand 2")

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax, label = r'RMSD ($\AA$)')

if save_verb: plt.savefig(results_path + 'rmsd/rmsd_map_lig.png', dpi=500, bbox_inches='tight')
if plot_verb:   
    plt.show()
else:
    plt.close()

## BINDING SITES

In [7]:
if run_analysis_verb:
    # take positions of only the binding site 1 CA atoms
    selection = binding_site1
    traj = []
    for ts in u.trajectory:
        traj.append(u.select_atoms(selection).positions)
    traj = np.array(traj)

    print("Now starting binding site 1 rmsd map calculation")

    with ProgressBar(total=len(traj)*len(traj)) as progress:
        rmsd_map_pot1 = compute_pairwise_rmsd(traj, traj, np.zeros((len(traj), len(traj))), progress)

    # take positions of only the binding site 2 CA atoms
    selection = binding_site2
    traj = []
    for ts in u.trajectory:
        traj.append(u.select_atoms(selection).positions)
    traj = np.array(traj)

    print("Now starting binding site 2 rmsd map calculation")

    with ProgressBar(total=len(traj)*len(traj)) as progress:
        rmsd_map_pot2 = compute_pairwise_rmsd(traj, traj, np.zeros((len(traj), len(traj))), progress)

    df = pd.DataFrame(rmsd_map_pot1).round(3)
    df.columns = [f'{i}' for i in range(nFrames)]
    df.to_parquet(data_path + 'rmsd_map/rmsd_pot1.parquet', index=False, engine = 'pyarrow')

    df = pd.DataFrame(rmsd_map_pot2).round(3)
    df.columns = [f'{i}' for i in range(nFrames)]
    df.to_parquet(data_path + 'rmsd_map/rmsd_pot2.parquet', index=False, engine = 'pyarrow')
else:
    try:
        rmsd_map_pot1 = pd.read_parquet(data_path + 'rmsd_map/rmsd_pot1.parquet')
        rmsd_map_pot2 = pd.read_parquet(data_path + 'rmsd_map/rmsd_pot2.parquet')
    except:
        print("No rmsd data found. Please run analysis first.")

In [ ]:
fig, (ax, ax1) = plt.subplots(1, 2, figsize=(10, 4))
im = ax.imshow(rmsd_map_pot1, cmap='viridis', vmin=0, vmax=7)
ax.set_xlabel('Time [ns]')
ax.set_ylabel('Time [ns]')
ax.set_xticks(ticks_arr, ticks_arr_new.astype(str))
ax.set_yticks(ticks_arr, ticks_arr_new.astype(str))
ax.set_title("Binding site 1")

im1 = ax1.imshow(rmsd_map_pot2, cmap='viridis', vmin=0, vmax=7)
ax1.set_xlabel('Time [ns]')
ax1.set_xticks(ticks_arr, ticks_arr_new.astype(str))
ax1.set_yticks(ticks_arr, ticks_arr_new.astype(str))
ax1.set_title("Binding site 2")

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax, label = r'RMSD ($\AA$)')

if save_verb: plt.savefig(results_path + 'rmsd/rmsd_map_pot.png', dpi=500, bbox_inches='tight')
if plot_verb:   
    plt.show()
else:
    plt.close()

## CHAINS RMSD MAP

In [8]:
if run_analysis_verb:
    # take positions of only the chain 1 CA atoms
    selection = 'name CA and (index 1:4690)'
    traj = []
    for ts in u.trajectory:
        traj.append(u.select_atoms(selection).positions)
    traj = np.array(traj)

    print("Now starting chain1 rmsd map calculation")

    with ProgressBar(total=len(traj)*len(traj)) as progress:
        rmsd_map_chain1 = compute_pairwise_rmsd(traj, traj, np.zeros((len(traj), len(traj))), progress)

    # take positions of only the chain 2 CA atoms
    selection = 'name CA and (index 4811:9500)'
    traj = []
    for ts in u.trajectory:
        traj.append(u.select_atoms(selection).positions)
    traj = np.array(traj)

    print("Now starting chain2 rmsd map calculation")

    with ProgressBar(total=len(traj)*len(traj)) as progress:
        rmsd_map_chain2 = compute_pairwise_rmsd(traj, traj, np.zeros((len(traj), len(traj))), progress)

    df = pd.DataFrame(rmsd_map_chain1).round(3)
    df.columns = [f'{i}' for i in range(nFrames)]
    df.to_parquet(data_path + 'rmsd_map/rmsd_chain1.parquet', index=False, engine = 'pyarrow')

    df = pd.DataFrame(rmsd_map_chain2).round(3)
    df.columns = [f'{i}' for i in range(nFrames)]
    df.to_parquet(data_path + 'rmsd_map/rmsd_chain2.parquet', index=False, engine = 'pyarrow')
else:
    try:
        rmsd_map_chain1 = pd.read_parquet(data_path + 'rmsd_map/rmsd_chain1.parquet')
        rmsd_map_chain2 = pd.read_parquet(data_path + 'rmsd_map/rmsd_chain2.parquet')
    except:
        print("No rmsd data found. Please run analysis first.")

In [ ]:
fig, (ax, ax1) = plt.subplots(1, 2, figsize=(10, 4))
im = ax.imshow(rmsd_map_chain1, cmap='viridis', vmin=0, vmax=4)
ax.set_xlabel('Time [ns]')
ax.set_ylabel('Time [ns]')
ax.set_xticks(ticks_arr, ticks_arr_new.astype(str))
ax.set_yticks(ticks_arr, ticks_arr_new.astype(str))
ax.set_title("Chain 1")

im1 = ax1.imshow(rmsd_map_chain2, cmap='viridis', vmin=0, vmax=4)
ax1.set_xlabel('Time [ns]')
ax1.set_xticks(ticks_arr, ticks_arr_new.astype(str))
ax1.set_yticks(ticks_arr, ticks_arr_new.astype(str))
ax1.set_title("Chain 2")

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax, label = r'RMSD ($\AA$)')

if save_verb: plt.savefig(results_path + 'rmsd/rmsd_map_chains.png', dpi=500, bbox_inches='tight')
if plot_verb:   
    plt.show()
else:
    plt.close()

## FULL CA RMSD MAP

In [9]:
if run_analysis_verb:
    selection = 'name CA'
    traj = []
    for ts in u.trajectory:
        traj.append(u.select_atoms(selection).positions)
    traj = np.array(traj)

    print("Now starting rmsd map calculation")

    with ProgressBar(total=len(traj)*len(traj)) as progress:
        rmsd_map_CA = compute_pairwise_rmsd(traj, traj, np.zeros((len(traj), len(traj))), progress)

    df = pd.DataFrame(rmsd_map_CA).round(3)
    df.columns = [f'{i}' for i in range(nFrames)]
    df.to_parquet(data_path + 'rmsd_map/rmsd_map_CA.parquet', index=False, engine = 'pyarrow')

else:
    try:
        rmsd_map_CA = pd.read_parquet(data_path + 'rmsd_map/rmsd_map_CA.parquet')
    except:
        print("No rmsd data found. Please run analysis first.")

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
im = ax.imshow(rmsd_map_CA, cmap='viridis')
ax.set_xlabel('Time [ns]')
ax.set_ylabel('Time [ns]')
ax.set_xticks(ticks_arr, ticks_arr_new.astype(str))
ax.set_yticks(ticks_arr, ticks_arr_new.astype(str))
fig.colorbar(im, label = r'RMSD ($\AA$)')
if save_verb: plt.savefig(results_path + 'rmsd/rmsd_map.png', dpi=500, bbox_inches='tight')
if plot_verb:
    plt.show()
else: plt.close()

# RMSF ANALYSIS

## BINDING SITE RMSF

In [ ]:
if run_analysis_verb:
    pot1_rmsf = rms.RMSF(binding_site1_CA, verbose = True).run().rmsf
    pot2_rmsf = rms.RMSF(binding_site2_CA, verbose = True).run().rmsf
    rmsf_pots = pd.DataFrame({'resid': binding_site1_CA.resids, 'pot1': pot1_rmsf, 'pot2': pot2_rmsf})
    rmsf_pots.to_csv(data_path+'rmsf_pots.csv', index=False, float_format='%.3f')
else:
    try: rmsf_pots = pd.read_csv(data_path+'rmsf_pots.csv')
    except: print('Error: You have to run the analysis for the first time')

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 4), sharex=True)
ax1.plot(binding_site1_CA.resids.astype(str), rmsf_pots.pot1)
ax1.set(ylabel = 'RMSF ($\AA$)', title = 'Binding site 1 RMSF')
ax2.plot(binding_site2_CA.resids.astype(str), rmsf_pots.pot2)
ax2.set(xlabel = 'Residue', ylabel = 'RMSF ($\AA$)', title = 'Binding site 2 RMSF')
ax1.grid()
ax2.grid()
plt.tight_layout()
if save_verb: plt.savefig(results_path+"rmsf/rmsf_binding_sites.png", dpi = 500, bbox_inches = 'tight')
if plot_verb: 
    plt.show()
else: plt.close()

## FULL CHAINS RMSF

In [ ]:
if run_analysis_verb:
    firstChain_rmsf = rms.RMSF(first_chain, verbose = True).run().rmsf
    secondChain_rmsf = rms.RMSF(second_chain, verbose = True).run().rmsf
    rmsf_chains = pd.DataFrame({'resid': first_chain.resids, 'chain1': firstChain_rmsf, 'chain2': secondChain_rmsf})
    rmsf_chains.to_csv(data_path+'rmsf_chains.csv', index=False, float_format='%.3f')
else:
    try: rmsf_chains = pd.read_csv(data_path + 'rmsf_chains.csv')
    except: print('Error: You have to run the analysis for the first time')

In [ ]:
arr = np.linspace(1, 306, 10, dtype=int)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6), sharex=True)
ax1.plot(first_chain.resids.astype(str), rmsf_chains.chain1)
ax1.set(ylabel = 'RMSF ($\AA$)', title = 'First Chain RMSF') 
ax1.set_xticks(arr, arr.astype(str))
ax1.grid()

for i in binding_site1_CA.resids: # red vertical lines for the binding site 
    if i == 144:
        ax1.vlines(i, min(rmsf_chains.chain1), max(rmsf_chains.chain1), color='red', linestyle = 'solid', alpha=1)
    else:
        ax1.vlines(i, min(rmsf_chains.chain1), max(rmsf_chains.chain1), color='black', linestyle = 'dotted', alpha=0.8)
if run==2: ax1.set_ylim(1, 6)

ax2.plot(second_chain.resids.astype(str), rmsf_chains.chain2)
ax2.set(xlabel = 'Residue', ylabel = 'RMSF ($\AA$)', title = 'Second Chain RMSF')
for i in binding_site2_CA.resids: # red vertical lines for the binding site 
    if i == 144:
        ax2.vlines(i, min(rmsf_chains.chain2), max(rmsf_chains.chain2), color='red', linestyle = 'solid', alpha=1)
    else:
        ax2.vlines(i, min(rmsf_chains.chain2), max(rmsf_chains.chain2), color='black', linestyle = 'dotted', alpha=0.8)
ax2.grid()

if run ==2: ax2.set_ylim(1, 6)

plt.tight_layout()
if save_verb: plt.savefig(results_path + "rmsf/rmsf.png", dpi = 500, bbox_inches = 'tight')
if plot_verb:
    plt.show()
else: plt.close()

In [ ]:
# version 2
arr = np.linspace(1, 306, 10, dtype=int)

fig, ax = plt.subplots(1, 1, figsize=(10, 3), sharex=True)
ax.plot(first_chain.resids.astype(str), rmsf_chains.chain1, label = 'Second Chain')
ax.set(ylabel = 'RMSF ($\AA$)', title = 'First Chain RMSF') 
ax.set_xticks(arr, arr.astype(str))
ax.grid()

for i in binding_site1_CA.resids: # red vertical lines for the binding site 
    if i == 144:
        ax.vlines(i, min(rmsf_chains.chain1), max(rmsf_chains.chain1), color='red', linestyle = 'solid', alpha=1)
    else:
        ax.vlines(i, min(rmsf_chains.chain1), max(rmsf_chains.chain1), color='black', linestyle = 'dotted', alpha=0.8)
if run==2: ax.set_ylim(1, 6)
ax.plot(second_chain.resids.astype(str), rmsf_chains.chain2, label = 'Second Chain')
if run ==2: ax2.set_ylim(1, 6)

ax.legend()
plt.tight_layout()
if save_verb: plt.savefig(results_path + "rmsf/rmsf_v2.png", dpi = 500, bbox_inches = 'tight')
if plot_verb:
    plt.show()
else: plt.close()

## RMSF BEFORE AND AFTER THE LIGAND DEPARTURE ??

## RMSF WINDOWED IN TIME

1 frame = 50 ps $\longrightarrow$ 1 ns = 20 frames


In [10]:
# nFrames/20 since we save coordinates every 50 ps
print(f"number of frames: {nFrames}, corresponding to {nFrames/20} ns" )
window = 500 # 25 ns, 25*20 = 500 frames
stride = 20 # 1 ns, 5*20 = 100 frames
print(f"window of {window/20} ns, stride of {stride/20} ns")

startFrames = np.arange(0, nFrames-window, stride, dtype=int)
endFrames = startFrames + window

number of frames: 10001, corresponding to 500.05 ns
window of 25.0 ns, stride of 1.0 ns


In [11]:
if run_analysis_verb:
    rmsf_list = []
    for i in tqdm(range(len(startFrames))):
        startFrame = startFrames[i]
        endFrame = endFrames[i]
        firstChain_rmsf = rms.RMSF(first_chain).run(startFrame, endFrame).rmsf
        secondChain_rmsf = rms.RMSF(second_chain).run(startFrame, endFrame).rmsf
        rmsf_list.append(pd.DataFrame({'resid': first_chain.resids, 'chain1': firstChain_rmsf, 'chain2': secondChain_rmsf}))

    rmsf_windowed = pd.concat(rmsf_list, axis=0, ignore_index=True)
    rmsf_windowed.to_csv(data_path + 'rmsf_windowed2.csv', index=False, float_format='%.3f')
else:
    try: rmsf_windowed = pd.read_csv(data_path + 'rmsf_windowed2.csv')
    except: print("Error: You need to run the analysis first")

# generate video with other environment running video.py, here only show
#Video(results_path+"rmsf/windowed_rmsf.mp4")

In [57]:
temp = np.empty((306, len(startFrames)))
for i in range(1, 307):
    temp[i-1] = np.array(rmsf_windowed.loc[rmsf_windowed.resid == i, "chain1"])

chain1_rmsf_heatmap = np.empty((306*2, len(startFrames)))
for i in range(0, 2*306, 2):
    chain1_rmsf_heatmap[i] = temp[int(i/2)]
    chain1_rmsf_heatmap[i+1] = temp[int(i/2)]

temp = np.empty((306, len(startFrames)))
for i in range(1, 307):
    temp[i-1] = np.array(rmsf_windowed.loc[rmsf_windowed.resid == i, "chain2"])

chain2_rmsf_heatmap = np.empty((306*2, len(startFrames)))
for i in range(0, 2*306, 2):
    chain2_rmsf_heatmap[i] = temp[int(i/2)]
    chain2_rmsf_heatmap[i+1] = temp[int(i/2)]

In [65]:
fig, (ax, ax1) = plt.subplots(1, 2, figsize = (12, 5), sharex=True, sharey=True)
im = ax.imshow(chain1_rmsf_heatmap.T, cmap='hot', interpolation='nearest', vmin = 1.5, vmax = 6)
ax.set(xlabel = 'Residues', ylabel = 'Window Time [ns]', title = 'First Chain RMSF')
ax.set_xticks(2*binding_site1_CA.resids, binding_site1_CA.resids.astype(str))
plt.setp(ax.get_xticklabels(), visible=False)

im2 = ax1.imshow(chain2_rmsf_heatmap.T, cmap='hot', interpolation='nearest', vmin = 1.5, vmax = 6)

ax1.set(xlabel = 'Residues', title = 'Second Chain RMSF')
ax1.set_xticks(2*binding_site1_CA.resids, binding_site1_CA.resids.astype(str))
plt.setp(ax1.get_xticklabels(), visible=False)

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax, label = r'RMSD ($\AA$)')
if save_verb: plt.savefig(results_path + "rmsf/rmsf_heatmap.png", dpi = 500, bbox_inches = 'tight')
if plot_verb:
    plt.show()
else: plt.close()

# LIGAND DISTANCE TO BINDING SITE

In [ ]:
if run_analysis_verb:
    d_lig1 = np.zeros(nFrames)
    d_lig2 = np.zeros(nFrames)
    for i, ts in tqdm(enumerate(u.trajectory)):
        d_lig1[i] = np.linalg.norm(binding_site1_CA.center_of_mass() - lig1_selection.center_of_mass())
        d_lig2[i] = np.linalg.norm(binding_site2_CA.center_of_mass() - lig2_selection.center_of_mass())
    d_ligands = pd.DataFrame({'lig1': d_lig1, 'lig2': d_lig2})
    d_ligands.to_csv(data_path+'ligands_distance.csv', index=False, float_format='%.3f')
else:
    try: d_ligands = pd.read_csv(data_path+'ligands_distance.csv')
    except: print("Error: You need to run the analysis first")

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6), sharex=True)
ax1.plot(d_ligands.index/20, d_ligands.lig1, '-b')
ax1.set(ylabel = 'Distance ($\AA$)', title = 'Ligand distance from corresponding binding site')
ax2.plot(d_ligands.index/20, d_ligands.lig2, '-r')
ax2.set(xlabel = 'Frame', ylabel = 'Distance ($\AA$)')
plt.tight_layout()
ax1.grid()
ax2.grid()
ax1.legend(['Ligand 1'])
ax2.legend(['Ligand 2'])
if save_verb: plt.savefig(results_path + "ligand_distance.png", dpi = 500, bbox_inches = 'tight')
if plot_verb: 
    plt.show()
else: plt.close()

# PLOT

In [ ]:
fig, (ax, ax1) = plt.subplots(1, 2, figsize=(12, 4), gridspec_kw={'width_ratios': [1.5, 1]})
ax.plot(rmsd_df.time, rmsd_df.CA)
ax.set(ylabel = 'RMSD [$\AA$]', xlabel = 'Time [ns]' , ylim = (1, 4), title = r'RMSD ($C_\alpha$)')
ax.grid()

im = ax1.imshow(rmsd_map_CA, cmap='viridis')
ax1.set_xlabel('Time [ns]')
ax1.set_ylabel('Time [ns]')
ax1.set_xticks(ticks_arr, ticks_arr_new.astype(str))
ax1.set_yticks(ticks_arr, ticks_arr_new.astype(str))
ax1.set_title(r"RMSD - Map ($C_\alpha$)")
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax, label = r'RMSD ($\AA$)')

if save_verb: plt.savefig(results_path + "rmsd/rmsd_CA.png", dpi = 500, bbox_inches = 'tight')
if plot_verb:
    plt.show()
else: plt.close()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 3), sharex=True)
ax.plot(rmsd_df.time, rmsd_df.lig1, label = 'Ligand 1', linewidth=0.8)
ax.plot(rmsd_df.time, rmsd_df.lig2, label = 'Ligand 2', linewidth=0.8)
ax.set(ylabel = 'RMSD [$\AA$]', xlabel = 'Time [ns]', title = 'Ligands - Root Mean Squared Displacement')
ax.legend()
ax.grid()

if save_verb: plt.savefig(results_path + "rmsd/rmsd_ligands.png", dpi = 500, bbox_inches = 'tight')
if plot_verb:
    plt.show()
else: plt.close()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 3), sharex=True)
ax.plot(rmsd_df.time, rmsd_df.pot1, label = 'Binding Site 1', linewidth=0.8)
ax.plot(rmsd_df.time, rmsd_df.pot2, label = 'Binding Site 2', linewidth=0.8)
ax.set(ylabel = 'RMSD [$\AA$]', xlabel = 'Time [ns]', title = 'Binding Sites - Root Mean Squared Displacement')
ax.legend()
ax.grid()

if save_verb: plt.savefig(results_path + "rmsd/rmsd_binding_sites.png", dpi = 500, bbox_inches = 'tight')
if plot_verb:
    plt.show()
else: plt.close()